In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import time

In [2]:
def encontrar_imoveis(df):
    pagina = 2
    endereço = []
    tamanho_m2 = []
    n_quartos = []
    valor_aluguel = []

    while pagina < 2500:
        html_text = requests.get(f'https://www.vivareal.com.br/aluguel/sp/sao-paulo/apartamento_residencial/?pagina={pagina}#onde=Brasil,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,,,,,BR%3ESao%20Paulo%3ENULL%3ESao%20Paulo,,,').text
        soup = BeautifulSoup(html_text, 'lxml')
        todos_imoveis = soup.find_all('article', class_='property-card__container js-property-card')
        time.sleep(2)
        
        #encontrando o endereço
        for i in todos_imoveis:
            nome_endereço = i.find('span', class_='property-card__address')
            if nome_endereço is not None:
                resultado = nome_endereço.get_text()
            else:
                resultado = -1
            endereço.append(resultado)
            
        #encontrando o tamanho do imóvel
        for i in todos_imoveis:
            m2 = i.find('span', class_='property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area')
            if m2 is not None:
                resultado = m2.get_text()
            else:
                resultado = 0
            tamanho_m2.append(resultado)
            
        #encontrando numero de quartos 
        for i in todos_imoveis:
            quartos = i.find('span', class_='property-card__detail-value js-property-card-value')
            if m2 is not None:
                resultado = quartos.get_text()
            else:
                resultado = 0
            n_quartos.append(resultado)
            
        #encontrando valor do aluguel
        for i in todos_imoveis:
            aluguel = i.find('p')
            if aluguel is not None:
                resultado = aluguel.get_text()
            else:
                resultado = 0
            valor_aluguel.append(resultado)        
        
        pagina = pagina + 1
    endereco_df = pd.DataFrame({'bairro':endereço, 'tamanho_m2':tamanho_m2,'n_quartos':n_quartos,'valor_aluguel':valor_aluguel})
    return endereco_df

In [3]:
df_raw_data = pd.DataFrame(columns=['bairro','tamanho_m2','n_quartos','valor_aluguel'])

In [4]:
df = encontrar_imoveis(df_raw_data)

In [5]:
#Funções para obter apenas o nome do bairro

def clean_bairros_p1(endereco_lista):
    new_list = []
    
    for end in endereco_lista:
        clean_p1 = end.rstrip('São Paulo - SP')
        clean_p1 = clean_p1.replace(',','')
        new_list.append(clean_p1)
    return new_list


def clean_bairros_p2(endereco_lista):
    new_list = []
    
    for endereco in endereco_lista:
        clean_end = endereco.split('-')
        
        if len(clean_end) == 1:
            new_list.append(clean_end[0])
        else:
            clean_end = clean_end[1].strip()
            new_list.append(clean_end)
    return new_list 

#Função para tirar os espaços do começo e do final da string
def clean_string(lista_tamanho):
    new_list = []
    for tamanho in lista_tamanho:
        tamanho = tamanho.strip()
        new_list.append(tamanho)
    return new_list

#Funções para deixar apenas o valor numerico do aluguel
def clean_aluguel_p1(text):
    text = text.str.replace(' /Mês','',regex=False)
    text = text.str.replace('R$ ', '',regex=False)
    text = text.str.replace('.', '',regex=False)
    return text  


def clean_aluguel_p2(text):
    text = text.str.replace(' Preço abaixo do mercado','',regex=False)
    return text  

In [6]:
df

,bairro,tamanho_m2,n_quartos,valor_aluguel
0,"Rua Ytaipu - Mirandópolis, São Paulo - SP",75,2,R$ 4.000 /Mês
1,"Rua do Cristal - Vila Mariana, São Paulo - SP",65,2,R$ 4.000 /Mês
2,"Cambuci, São Paulo - SP",54,2,R$ 2.550 /Mês
3,"Rua da Independência, 706 - Cambuci, São Paulo...",98,3,R$ 5.000 /Mês
4,"Jardim Jaú, São Paulo - SP",60,2,R$ 1.500 /Mês
...,...,...,...,...
86683,"Avenida Miguel Yunes, 540 - Usina Piratininga,...",70,3,R$ 3.200 /Mês
86684,"Vila Nova Manchester, São Paulo - SP",78,3,R$ 2.500 /Mês
86685,"Rua Melo Alves, 265 - Cerqueira César, São Pau...",120,2,R$ 4.000 /Mês
86686,"Rua Coronel Cabrita - Jardim da Glória, São Pa...",124,3,R$ 6.700 /Mês


In [7]:
df_clean_data = df.copy()

In [8]:
df_clean_data['bairro'] = clean_bairros_p1(df_clean_data['bairro'])
df_clean_data['bairro'] = clean_bairros_p2(df_clean_data['bairro'])

In [9]:
df_clean_data['tamanho_m2'] = clean_string(df_clean_data['tamanho_m2'])

In [10]:
df_clean_data['n_quartos'] = clean_string(df_clean_data['n_quartos'])

In [11]:
df_clean_data['valor_aluguel'] = clean_aluguel_p1(df_clean_data['valor_aluguel'])
df_clean_data['valor_aluguel'] = clean_aluguel_p2(df_clean_data['valor_aluguel'])

In [12]:
df_clean_data

,bairro,tamanho_m2,n_quartos,valor_aluguel
0,Mirandópolis,75,2,4000
1,Vila Mariana,65,2,4000
2,Cambuci,54,2,2550
3,Cambuci,98,3,5000
4,Jardim Jaú,60,2,1500
...,...,...,...,...
86683,Usina Piratininga,70,3,3200
86684,Vila Nova Manchester,78,3,2500
86685,Cerqueira César,120,2,4000
86686,Jardim da Glória,124,3,6700


In [13]:
df_clean_data.dtypes

bairro           object
tamanho_m2       object
n_quartos        object
valor_aluguel    object
dtype: object

In [14]:
df_clean_data['tamanho_m2'] = df_clean_data['tamanho_m2'].astype(float)
df_clean_data['n_quartos'] = df_clean_data['n_quartos'].astype(float)
df_clean_data['valor_aluguel'] = df_clean_data['valor_aluguel'].astype(float)

In [15]:
df_clean_data.dtypes

bairro            object
tamanho_m2       float64
n_quartos        float64
valor_aluguel    float64
dtype: object

In [24]:
df_clean_data.to_parquet('df_imoveis_VivaReal.parquet')